In [40]:
src_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
src_zip = "dakshina_dataset_v1.0.tar"
DATA_SRC="dakshina_dataset_v1.0/ta/lexicons"
DATA_TRAIN_SRC = "/ta.translit.sampled.train.tsv"
DATA_VAL_SRC = "/ta.translit.sampled.dev.tsv" 
DATA_TEST_SRC = "/ta.translit.sampled.test.tsv"
#TRAIN_IMAGES_PER_LABEL = 1000
#TEST_IMAGES_PER_LABEL = 200
BALANCED_SPLITS = {"train" : 900, "val" : 100}
PROJECT_NAME = "CS6910 ASSIGNMENT 3"
dataset='dakshina-dataset'

In [ ]:
%%capture
!curl -SL $src_url > $src_zip
!tar -xf $src_zip

In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd
import keras

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

wandb: Currently logged in as: cs6910krsrd (use `wandb login --relogin` to force relogin)


Uploading Data: only run once

In [ ]:

run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(dataset, type="raw_data")

raw_data_at.add_dir(DATA_SRC)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

Downloading Data

In [42]:
run = wandb.init(project=PROJECT_NAME, entity='cs6910krsrd',job_type="download")

# Query W&B for an artifact and mark it as input to this run
artifact = run.use_artifact(dataset+':latest')

# Download the artifact's contents
artifact_dir = artifact.download()
run.finish()

In [43]:
colnames=["ntv","rmn",'nAtt']
df_train = pd.read_csv(artifact_dir + DATA_TRAIN_SRC,sep="\t",names=colnames,na_filter=False)
df_val = pd.read_csv(artifact_dir + DATA_VAL_SRC,sep="\t",names=colnames,na_filter=False)

In [44]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [45]:
input_texts = df_train.ntv.to_list()
target_texts = df_train.rmn.apply(lambda s:'\t'+s+'\n').to_list()
input_characters = set(df_train.ntv.sum())
target_characters = set(df_train.rmn.sum())


input_characters = sorted(list(input_characters))
input_characters.append(' ')
target_characters = sorted(list(target_characters))
target_characters.append(' ')
target_characters.append('\t')
target_characters.append('\n')



num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [46]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)



Number of samples: 68218
Number of unique input tokens: 47
Number of unique output tokens: 29
Max sequence length for inputs: 26
Max sequence length for outputs: 32


In [47]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [48]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
853/853 [==============================] - 11s 10ms/step - loss: 0.9456 - accuracy: 0.7324 - val_loss: 0.7348 - val_accuracy: 0.7916
Epoch 2/100
853/853 [==============================] - 8s 9ms/step - loss: 0.5477 - accuracy: 0.8240 - val_loss: 0.6855 - val_accuracy: 0.8035
Epoch 3/100
853/853 [==============================] - 8s 9ms/step - loss: 0.4349 - accuracy: 0.8618 - val_loss: 0.6201 - val_accuracy: 0.8218
Epoch 4/100
853/853 [==============================] - 8s 9ms/step - loss: 0.3239 - accuracy: 0.8983 - val_loss: 0.5566 - val_accuracy: 0.8449
Epoch 5/100
853/853 [==============================] - 8s 9ms/step - loss: 0.2464 - accuracy: 0.9224 - val_loss: 0.5027 - val_accuracy: 0.8585
Epoch 6/100
853/853 [==============================] - 8s 9ms/step - loss: 0.2017 - accuracy: 0.9356 - val_loss: 0.4648 - val_accuracy: 0.8676
Epoch 7/100
853/853 [==============================] - 8s 10ms/step - loss: 0.1736 - accuracy: 0.9432 - val_loss: 0.4653 - val_accuracy: 0.8